<a href="https://colab.research.google.com/github/j-buss/wi-dpi-analysis/blob/development/eda/6.1_Salary_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Salary and Education in Wisconsin - 6.1 Salary Prediction


Salary prediction for all percentage of employees

### Import Libraries

In [0]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
import seaborn as sns
import matplotlib.pyplot as plt

from google.cloud import bigquery

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [0]:
%matplotlib inline
plt.style.use('bmh')

### Functions

In [0]:
# To help with the pie charts later
def make_autopct(values):
  def my_autopct(pct):
    total = sum(values)
    val = int(round(pct*total/100.0))
    return '{p:.2f}% \n({v:,d})'.format(p=pct,v=val)
  return my_autopct

### Processing

In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

In [0]:
project_id='wi-dpi-010'
landing_dataset_name='landing'
refined_dataset_name='refined'
gold_dataset_name='gold'

In [0]:
bq_client = bigquery.Client(project=project_id)

## Analysis


### How Many People Have a Salary?

In [0]:
select_people_w_salary = '''
SELECT
  count(distinct research_id)
FROM
  {}.{}
WHERE
  salary is not null
'''.format(refined_dataset_name, '2017_all_staff_report')

### Probability density for # of Records

In [0]:
select_2017 = '''
SELECT
  research_id,
  gender,
  race_ethnicity_cd,
  birth_year,
  contract_high_degree_cd,
  contract_days,
  contract_local_experience,
  contract_total_experience,
  cesa_num,
  
  assignment_fte,
  assignment_requires_dpi_license,
  assignment_staff_category_cd,
  position_classification,
  assignment_position_cd,
  assignment_area_cd,
  assignment_work_school_level_cd,
  assignment_work_agency_cd,
  salary
FROM
  {}.{}
WHERE
  assignment_fte = 1 and salary is not null
'''.format(refined_dataset_name, '2017_all_staff_report')

In [10]:
df = pd.read_gbq(select_2017, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55633 entries, 0 to 55632
Data columns (total 18 columns):
research_id                        55633 non-null int64
gender                             55633 non-null object
race_ethnicity_cd                  55633 non-null object
birth_year                         55633 non-null int64
contract_high_degree_cd            55628 non-null object
contract_days                      55633 non-null int64
contract_local_experience          55633 non-null float64
contract_total_experience          55633 non-null float64
cesa_num                           55572 non-null object
assignment_fte                     55633 non-null float64
assignment_requires_dpi_license    55633 non-null object
assignment_staff_category_cd       55633 non-null object
position_classification            55633 non-null object
assignment_position_cd             55633 non-null object
assignment_area_cd                 55633 non-null object
assignment_work_school_level_cd    5

In [12]:
df.head(10)

,research_id,gender,race_ethnicity_cd,birth_year,contract_high_degree_cd,contract_days,contract_local_experience,contract_total_experience,cesa_num,assignment_fte,assignment_requires_dpi_license,assignment_staff_category_cd,position_classification,assignment_position_cd,assignment_area_cd,assignment_work_school_level_cd,assignment_work_agency_cd,salary
0,47079,F,W,1959,5,260,2.0,36.0,10,1.0,Y,1,Administrators,05,0000,None,0007,108840.0
1,66962,M,W,1972,5,260,1.0,1.0,5,1.0,Y,1,Administrators,05,0000,None,0014,118000.0
2,29934,F,W,1971,5,260,2.0,20.0,2,1.0,Y,1,Administrators,05,0000,None,0063,92000.0
3,41977,M,W,1972,5,240,8.0,21.0,4,1.0,Y,1,Administrators,05,0000,None,0091,109629.0
4,21817,F,W,1953,7,260,6.0,6.0,10,1.0,Y,1,Administrators,05,0000,None,0112,136331.0
5,66608,M,W,1963,6,261,3.0,31.0,11,1.0,Y,1,Administrators,05,0000,None,0119,145550.0
6,53446,F,W,1967,7,261,1.0,21.0,9,1.0,Y,1,Administrators,05,0000,None,0140,150000.0
7,50365,F,W,1957,7,261,23.0,40.0,6,1.0,Y,1,Administrators,05,0000,None,0147,185000.0
8,90990,M,W,1960,5,260,19.0,34.0,4,1.0,Y,1,Administrators,05,0000,None,0154,124258.0
9,22531,M,W,1965,7,260,8.0,8.0,12,1.0,Y,1,Administrators,05,0000,None,0170,145030.0


### Predict Salary for Full Time Teachers in Only One Role

#### Version 1

In [0]:
df_v1 = df[['gender','birth_year','contract_total_experience','salary']]
df_v1 = pd.concat([df_v1.drop('gender', axis=1), pd.get_dummies(df_v1['gender'],prefix='gender')], axis=1)

In [48]:
df_v1.head()

,birth_year,contract_total_experience,salary,gender_F,gender_M
0,1959,36.0,108840.0,1,0
1,1972,1.0,118000.0,0,1
2,1971,20.0,92000.0,1,0
3,1972,21.0,109629.0,0,1
4,1953,6.0,136331.0,1,0


In [0]:
target_v1 = df_v1['salary']
data_v1 = df_v1.drop('salary', axis=1)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(data_v1, target_v1, test_size=0.25, random_state=0)
model_v1 = LinearRegression()
model_v1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [51]:
model_v1.score(X_test, y_test)

0.368015985502004

#### Version 2

In [0]:
df_v2 = df.copy()
df_v2 = df_v2.drop(['research_id','assignment_requires_dpi_license','assignment_position_cd','assignment_area_cd','assignment_work_agency_cd'], axis=1)

In [0]:

df_v2 = pd.concat([df_v2.drop('gender', axis=1), pd.get_dummies(df_v2['gender'],prefix='gender')], axis=1)
df_v2 = pd.concat([df_v2.drop('race_ethnicity_cd', axis=1), pd.get_dummies(df_v2['race_ethnicity_cd'],prefix='ethnicity')], axis=1)
df_v2 = pd.concat([df_v2.drop('contract_high_degree_cd', axis=1), pd.get_dummies(df_v2['contract_high_degree_cd'],prefix='degree')], axis=1)
df_v2 = pd.concat([df_v2.drop('cesa_num', axis=1), pd.get_dummies(df_v2['cesa_num'],prefix='cesa')], axis=1)
df_v2 = pd.concat([df_v2.drop('assignment_staff_category_cd', axis=1), pd.get_dummies(df_v2['assignment_staff_category_cd'],prefix='assignment_staff_category_cd')], axis=1)
df_v2 = pd.concat([df_v2.drop('position_classification', axis=1), pd.get_dummies(df_v2['position_classification'])], axis=1)
df_v2 = pd.concat([df_v2.drop('assignment_work_school_level_cd', axis=1), pd.get_dummies(df_v2['assignment_work_school_level_cd'],prefix='assignment_work_school_level_cd')], axis=1)

In [0]:
df_v2.columns = df_v2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [55]:
df_v2.head()

,birth_year,contract_days,contract_local_experience,contract_total_experience,assignment_fte,salary,gender_f,gender_m,ethnicity_a,ethnicity_b,ethnicity_h,ethnicity_i,ethnicity_p,ethnicity_t,ethnicity_w,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8,cesa_1,cesa_2,cesa_3,cesa_4,cesa_5,cesa_6,cesa_7,cesa_8,cesa_9,cesa_10,cesa_11,cesa_12,assignment_staff_category_cd_0,assignment_staff_category_cd_1,assignment_staff_category_cd_3,administrators,other,pupil_services,teachers,assignment_work_school_level_cd_3,assignment_work_school_level_cd_4,assignment_work_school_level_cd_5,assignment_work_school_level_cd_6,assignment_work_school_level_cd_7
0,1959,260,2.0,36.0,1.0,108840.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0
1,1972,260,1.0,1.0,1.0,118000.0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
2,1971,260,2.0,20.0,1.0,92000.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
3,1972,240,8.0,21.0,1.0,109629.0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
4,1953,260,6.0,6.0,1.0,136331.0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0


In [0]:
target_v2 = df_v2['salary']
data_v2 = df_v2.drop('salary', axis=1)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(data_v2, target_v2, test_size=0.2, random_state=0)
model_v2 = LinearRegression()
model_v2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [58]:
model_v2.score(X_test, y_test)

0.7967505594073234

### Version 3

In [59]:
df.head(10)

,research_id,gender,race_ethnicity_cd,birth_year,contract_high_degree_cd,contract_days,contract_local_experience,contract_total_experience,cesa_num,assignment_fte,assignment_requires_dpi_license,assignment_staff_category_cd,position_classification,assignment_position_cd,assignment_area_cd,assignment_work_school_level_cd,assignment_work_agency_cd,salary
0,47079,F,W,1959,5,260,2.0,36.0,10,1.0,Y,1,Administrators,05,0000,None,0007,108840.0
1,66962,M,W,1972,5,260,1.0,1.0,5,1.0,Y,1,Administrators,05,0000,None,0014,118000.0
2,29934,F,W,1971,5,260,2.0,20.0,2,1.0,Y,1,Administrators,05,0000,None,0063,92000.0
3,41977,M,W,1972,5,240,8.0,21.0,4,1.0,Y,1,Administrators,05,0000,None,0091,109629.0
4,21817,F,W,1953,7,260,6.0,6.0,10,1.0,Y,1,Administrators,05,0000,None,0112,136331.0
5,66608,M,W,1963,6,261,3.0,31.0,11,1.0,Y,1,Administrators,05,0000,None,0119,145550.0
6,53446,F,W,1967,7,261,1.0,21.0,9,1.0,Y,1,Administrators,05,0000,None,0140,150000.0
7,50365,F,W,1957,7,261,23.0,40.0,6,1.0,Y,1,Administrators,05,0000,None,0147,185000.0
8,90990,M,W,1960,5,260,19.0,34.0,4,1.0,Y,1,Administrators,05,0000,None,0154,124258.0
9,22531,M,W,1965,7,260,8.0,8.0,12,1.0,Y,1,Administrators,05,0000,None,0170,145030.0


In [0]:
df_v3 = df.copy()
df_v3['assignment_requires_dpi_license'] = df_v3['assignment_requires_dpi_license'].replace(to_replace={'Y':1, 'N':0})
df_v3 = df_v3.drop(['research_id'], axis=1)

In [0]:
df_v3 = pd.concat([df_v3.drop('gender', axis=1), pd.get_dummies(df_v3['gender'],prefix='gender')], axis=1)
df_v3 = pd.concat([df_v3.drop('race_ethnicity_cd', axis=1), pd.get_dummies(df_v3['race_ethnicity_cd'],prefix='ethnicity')], axis=1)
df_v3 = pd.concat([df_v3.drop('contract_high_degree_cd', axis=1), pd.get_dummies(df_v3['contract_high_degree_cd'],prefix='degree')], axis=1)
df_v3 = pd.concat([df_v3.drop('cesa_num', axis=1), pd.get_dummies(df_v3['cesa_num'],prefix='cesa')], axis=1)
df_v3 = pd.concat([df_v3.drop('assignment_staff_category_cd', axis=1), pd.get_dummies(df_v3['assignment_staff_category_cd'],prefix='assignment_staff_category_cd')], axis=1)
df_v3 = pd.concat([df_v3.drop('position_classification', axis=1), pd.get_dummies(df_v3['position_classification'])], axis=1)
df_v3 = pd.concat([df_v3.drop('assignment_work_school_level_cd', axis=1), pd.get_dummies(df_v3['assignment_work_school_level_cd'],prefix='assignment_work_school_level_cd')], axis=1)

df_v3 = pd.concat([df_v3.drop('assignment_position_cd', axis=1), pd.get_dummies(df_v3['assignment_position_cd'],prefix='assignment_position_cd')], axis=1)
df_v3 = pd.concat([df_v3.drop('assignment_work_agency_cd', axis=1), pd.get_dummies(df_v3['assignment_work_agency_cd'],prefix='assignment_work_agency_cd')], axis=1)
df_v3 = df_v3.drop('assignment_area_cd', axis=1)

In [0]:
df_v3.columns = df_v3.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [63]:
df_v3.head(10)

,birth_year,contract_days,contract_local_experience,contract_total_experience,assignment_fte,assignment_requires_dpi_license,salary,gender_f,gender_m,ethnicity_a,ethnicity_b,ethnicity_h,ethnicity_i,ethnicity_p,ethnicity_t,ethnicity_w,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8,cesa_1,cesa_2,cesa_3,cesa_4,cesa_5,cesa_6,cesa_7,cesa_8,cesa_9,cesa_10,cesa_11,cesa_12,assignment_staff_category_cd_0,assignment_staff_category_cd_1,assignment_staff_category_cd_3,administrators,other,pupil_services,teachers,assignment_work_school_level_cd_3,assignment_work_school_level_cd_4,assignment_work_school_level_cd_5,assignment_work_school_level_cd_6,assignment_work_school_level_cd_7,assignment_position_cd_05,assignment_position_cd_06,assignment_position_cd_08,assignment_position_cd_09,assignment_position_cd_10,assignment_position_cd_17,assignment_position_cd_18,assignment_position_cd_19,assignment_position_cd_50,assignment_position_cd_51,assignment_position_cd_52,assignment_position_cd_53,assignment_position_cd_54,assignment_position_cd_55,assignment_position_cd_59,assignment_position_cd_62,assignment_position_cd_63,assignment_position_cd_64,assignment_position_cd_73,assignment_position_cd_75,assignment_position_cd_79,assignment_position_cd_80,assignment_position_cd_83,assignment_position_cd_84,assignment_position_cd_85,assignment_position_cd_86,assignment_position_cd_87,assignment_position_cd_88,assignment_position_cd_90,assignment_position_cd_91,assignment_position_cd_92,assignment_position_cd_93,assignment_position_cd_98,assignment_position_cd_99,assignment_work_agency_cd_0007,assignment_work_agency_cd_0014,assignment_work_agency_cd_0063,assignment_work_agency_cd_0070,assignment_work_agency_cd_0084,assignment_work_agency_cd_0091,assignment_work_agency_cd_0105,assignment_work_agency_cd_0112,assignment_work_agency_cd_0119,assignment_work_agency_cd_0126,assignment_work_agency_cd_0140,assignment_work_agency_cd_0147,assignment_work_agency_cd_0154,assignment_work_agency_cd_0161,assignment_work_agency_cd_0170,assignment_work_agency_cd_0182,assignment_work_agency_cd_0196,assignment_work_agency_cd_0203,assignment_work_agency_cd_0217,assignment_work_agency_cd_0231,assignment_work_agency_cd_0238,assignment_work_agency_cd_0245,assignment_work_agency_cd_0280,assignment_work_agency_cd_0287,assignment_work_agency_cd_0308,assignment_work_agency_cd_0315,assignment_work_agency_cd_0336,assignment_work_agency_cd_0350,assignment_work_agency_cd_0364,assignment_work_agency_cd_0413,assignment_work_agency_cd_0422,assignment_work_agency_cd_0427,assignment_work_agency_cd_0434,assignment_work_agency_cd_0441,assignment_work_agency_cd_0469,assignment_work_agency_cd_0476,assignment_work_agency_cd_0485,assignment_work_agency_cd_0490,assignment_work_agency_cd_0497,assignment_work_agency_cd_0602,assignment_work_agency_cd_0609,assignment_work_agency_cd_0616,assignment_work_agency_cd_0623,assignment_work_agency_cd_0637,assignment_work_agency_cd_0657,assignment_work_agency_cd_0658,assignment_work_agency_cd_0665,assignment_work_agency_cd_0700,assignment_work_agency_cd_0714,assignment_work_agency_cd_0721,assignment_work_agency_cd_0735,assignment_work_agency_cd_0777,assignment_work_agency_cd_0840,assignment_work_agency_cd_0870,assignment_work_agency_cd_0882,assignment_work_agency_cd_0896,assignment_work_agency_cd_0903,assignment_work_agency_cd_0910,assignment_work_agency_cd_0980,assignment_work_agency_cd_0994,assignment_work_agency_cd_1015,assignment_work_agency_cd_1029,assignment_work_agency_cd_1071,assignment_work_agency_cd_1080,assignment_work_agency_cd_1085,assignment_work_agency_cd_1092,assignment_work_agency_cd_1120,assignment_work_agency_cd_1127,assignment_work_agency_cd_1134,assignment_work_agency_cd_1141,assignment_work_agency_cd_1155,assignment_work_agency_cd_1162,assignment_work_agency_cd_1169,assignment_work_agency_cd_1176,assignment_work_agency_cd_1183,assignment_work_agency_cd_1204,assignment_work_agency_cd_1218,assignment_work_agency_cd_1232,assignment_work

In [0]:
target_v3 = df_v3['salary']
data_v3 = df_v3.drop('salary', axis=1)

In [65]:
X_train, X_test, y_train, y_test = train_test_split(data_v3, target_v3, test_size=0.2, random_state=0)
model_v3 = LinearRegression()
model_v3.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [66]:
model_v3.score(X_test, y_test)

0.8549880829480746

### Test a new Point

In [0]:
test_point = [[1969,15.0,0,1]]

In [74]:
model_v1.predict(test_point)

array([64828.08996283])

### NEXT FEW CELLS SHOULD PROBABLY GO INTO 3.0 Refined Notebook

### New Dataset - Teacher Master Data

|Query Grp.|Role Count | Contract Count | Record Count | Educator Count|
|---------|------|----|------------------------|--------------------------|
|A| 1 | 1 | 114,00|114,000|
|B| Mult. | 1 | 1 | 1 |
|C|Mult. | Mult. |2|2|
|Z|||187,344|143,493|

Hypothesis: Contract Days are tied to a Salary; Therefore if you group by contract days you should not have multiple salaries for contract days for a person

In [0]:
select_grp_A = '''
SELECT
  count(*)
FROM
  (SELECT
    research_id,
    first_name,
    last_name,
    count(*)
  FROM
    `wi-dpi-010.refined.2017_all_staff_report`
  GROUP BY
    1,2,3
  HAVING count(*) = 1) A

'''

In [0]:
select_grp_B_rec_cnt = '''
'''

In [0]:
select_grp_Z_educ_cnt = '''
SELECT
  count(distinct research_id)
FROM
  `wi-dpi-010.refined.2017_all_staff_report`
'''

In [0]:
# Every role for the educator has a salary and contract_days
#    This applies even if they have multiple roles
select_salary_educators = '''
'''

In [0]:
# MIXED Educators who have at least one salaried role and 
#    at least one 'Non-Salaried' role
select_mixed_educ_cnt ='''
SELECT
  count(distinct A.research_id)
FROM
  `wi-dpi-010.refined.2017_all_staff_report` A
  INNER JOIN `wi-dpi-010.refined.2017_all_staff_report` B ON
  A.research_id = B.research_id
WHERE 
  A.salary is not null AND
  B.salary is null
'''

In [0]:
# MIXED Educators who have at least one salaried role and 
#    at least one 'Non-Salaried' role
#    This count is essentially using the previous query as subselect
#    then selecting all associated records
select_mixed_rec_cnt='''
SELECT
  count(*)
FROM
  (SELECT
    distinct A.research_id
  FROM
    `wi-dpi-010.refined.2017_all_staff_report` A
    INNER JOIN `wi-dpi-010.refined.2017_all_staff_report` B ON
    A.research_id = B.research_id
  WHERE 
    A.salary is not null AND
    B.salary is null) C 
  INNER JOIN `wi-dpi-010.refined.2017_all_staff_report` D ON
  C.research_id = D.research_id
'''

### Next Data Set

In [0]:
select_teacher_master_data = '''
SELECT
  research_id,
  year_session,
  first_name,
  last_name,
  gender,
  race_ethnicity_cd,
  birth_year,
  contract_days,
  
  contract_high_degree_cd,
  contract_days,
  contract_local_experience,
  contract_total_experience,
  
  cesa_num,
  assignment_fte,
  assignment_requires_dpi_license,
  assignment_staff_category_cd,
  position_classification,
  assignment_position_cd,
  assignment_area_cd,
  assignment_work_school_level_cd,
  assignment_work_agency_cd,
  salary
FROM
  {}.{}
WHERE
  assignment_fte is not null and salary is not null
'''.format(refined_dataset_name, '2017_all_staff_report')

In [0]:
df = pd.read_gbq(select_all_salaries, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [0]:
assignment_std = df['assignment_fte'].std()
assignment_mean = df['assignment_fte'].mean()

In [0]:
df['assign_fte_1_std'] = 

### Host a model

Evaluation: https://www.ritchieng.com/machine-learning-evaluate-linear-regression-model/

ML Engine - https://cloud.google.com/ml-engine/docs/scikit/using-pipelines

### Extra Queries

In [0]:
Query_for_Sum_of_FTE='''
SELECT
  A.assgn_fte,
  count(a.research_id)
FROM
  (SELECT
    research_id,
    count(*) as person_assgn_cnt,
    round(sum(assignment_fte),2) as assgn_fte
  FROM
    refined.2017_all_staff_report
  WHERE
    salary is not null
  GROUP BY
    research_id) A
GROUP BY a.assgn_fte
ORDER BY 2 DESC'''

In [0]:
'''
SELECT
  research_id,
  year_session,
  first_name,
  last_name,
  gender,
  race_ethnicity_cd,
  birth_year,
  contract_days,
  contract_local_experience, 
  contract_total_experience,
  salary,
  cesa_num,
  assignment_fte,
  assignment_requires_dpi_license,
  assignment_staff_category_cd,
  position_classification,
  assignment_position_cd,
  assignment_area_cd,
  assignment_work_school_level_cd
  assignment_work_agency_cd
FROM
  `wi-dpi-010.refined.2017_all_staff_report`
WHERE
  salary is not null and assignment_fte is not null
ORDER BY
  research_id,
  assignment_fte desc'''